### Written Report for CMPT 810

This notebook shows an example of using the heat method (Crane et al., 2012. https://www.cs.cmu.edu/~kmcrane/Projects/HeatMethod/) on triangular meshes. I used trimesh


### Heat Method Algorithm

The Algorithm has only three steps. The idea behind this method comes from Varadhan's formula where he showed the relationship between heat and distance. The problem with directly using this formula is that approximating the heat kernel gives poor results for the distance. The authors have noticed that the gradients of the heat kernel point to the same direction, but with different magnitude. They used these gradients for recovering the distance.

1. Solve the heat equation $u$ for some fixed time $t$
2. Compute the gradient, $\nabla u$, and normalize the gradient, $X$
3. Solve the Poisson equation to recover the distance, $\phi$

![heat method](Steps.png)

The image above is a visualization of the steps. Image is from 
https://www.cs.cmu.edu/~kmcrane/Projects/HeatMethod/paperCACM.pdf

### Simon the Cat : Example using Triangle Meshes

#### Import the 3D Model

In [1]:
%matplotlib inline
import trimesh
import numpy as np
import geodesic_distance as gd

In [2]:
# from http://3dmag.org/en/market/item/5825/ 
mesh = trimesh.load('simon-cat-hungry.stl')

polys = np.array(mesh.faces)
pts = np.array(mesh.vertices)
pts = pts.astype(np.double)

#### Visualize Model

In [3]:
mesh.show()

/home/blanche/anaconda3/envs/algo/lib/python3.6/site-packages/IPython/core/display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


#### Compute Geodesic Distance from a Source Point

Note that the source point is colored in red. 

In [4]:
this_point = 128

geo_dist = gd.geodesic_distance(pts, polys, this_point)

color_this = gd.color_geo(geo_dist)

mesh.visual.vertex_colors = color_this
mesh.show()

/home/blanche/.local/lib/python3.6/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:296: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


#### With Obstacles in the Mesh

The obstacle is shown in black.

In [5]:
# Apply some obstacles

check_num = []
no = 0
for i in pts:
    if (26 <= i[0] <= 30) and (-20 <= i[1] <= 0) and (30 <= i[2] <= 40):
        check_num.append(no)
    no += 1
    

new_polys = []
for i in polys:
    if len([el for el in check_num if el in i]) == 0:
        new_polys.append(i)
        
new_polys = np.array(new_polys)

In [6]:
# Recompute the geodesic distance with obstacles
this_point = 128
geo_dist1 = gd.geodesic_distance(pts, new_polys, this_point)

color_this1 = gd.color_geo(geo_dist1)

mesh.visual.vertex_colors = color_this1

mesh.show()

#### Compare the distances with and without obstacles

The vertex (and the faces with this vertex) with the minimum and maximum difference from the distance without obstacles are colored in blue.

In [7]:
diff = geo_dist - geo_dist1

# set to 0 the difference for the vertices in obstacles
diff[check_num] = 0

print(diff.min(), diff.max())

-10.163157978680623 2.353080392509014


In [8]:
max_diff = np.argmax(diff)

new_color = np.copy(color_this1)
new_color[max_diff] = [0, 0, 255, 0]
mesh.visual.vertex_colors = new_color

mesh.show()

In [9]:
min_diff = np.argmin(diff)

new_color = np.copy(color_this1)
new_color[min_diff] = [0, 0, 255, 0]
mesh.visual.vertex_colors = new_color

mesh.show()